In [8]:
import pennylane as qml
import pennylane.numpy as np
import torch
from torch.nn import MSELoss, CrossEntropyLoss
from torch.optim import Adam, Adagrad
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split


import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [9]:
EPOCHS = 10

In [10]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev, interface='torch')
# TODO: use QCNN circuit instead
def QCNN_circuit(params, data:torch.Tensor):
    qml.AmplitudeEmbedding(data.flatten()[:4], wires=(0, 1), pad_with=0, normalize=True)
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.PhaseShift(params[2], wires=0)
    return qml.expval(qml.PauliZ(0))

In [11]:
from pandas import read_csv
class MyDataset(Dataset):
 
  def __init__(self,file_name):
    price_df=read_csv(file_name)

    x=price_df.iloc[:,0:8].values
    y=price_df.iloc[:,8].values

    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [ ]:
# QCNN Circuit

def qc(thetas, ansatz_f, ansatz_param, pol_f, pol_param, data):
    # insert initial state as data in n_qubits, not total_qubits
    qml.QubitStateVector(data, wires=range(self.n_qubits))
    theta_idx = 0
    for layer in range(self.total_layer):
        qubit_info = np.array(self.QCNN_tree[layer])
        L = len(qubit_info)
        qubit_info_index = np.array(range(L))
        if self.stride == 1:
            qubit_info_splited = [[idx for idx in qubit_info_index if idx % 2 == 0], [
                idx for idx in qubit_info_index if idx % 2 == 1]]
        else:
            qubit_info_splited = qubit_info_index
        # apply unitary
        for index in qubit_info_splited:
            for idx in index:
                if qubit_info[idx] != qubit_info[(idx+self.stride) % L]:
                    ansatz_f(
                        thetas[theta_idx:theta_idx+ansatz_param], wires=[qubit_info[idx], qubit_info[(idx+self.stride) % L]])
                    theta_idx += ansatz_param
        # apply pooling
        for idx in qubit_info_index:
            if idx % 2 == 0:
                if qubit_info[idx] != qubit_info[(idx+1) % L]:
                    pol_f(thetas[theta_idx:theta_idx+pol_param],
                            wires=[qubit_info[idx], qubit_info[(idx+1) % L]])
                    theta_idx += pol_param
    return qml.expval(qml.PauliZ(0))

In [12]:

# TODO: use postprocessed data instead
# ====================================
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & test, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
training_set, validation_set = random_split(training_set, [50000, 10000])
test_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for test
training_loader = DataLoader(training_set, batch_size=4, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_set, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

# ====================================

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('test set has {} instances'.format(len(test_set)))

Training set has 50000 instances
test set has 10000 instances


In [6]:
loss_fn = MSELoss() # TODO: choose loss function
params = torch.rand(3, requires_grad=True) # TODO: choose params length

optimizer = Adam([params], lr=0.01) # TODO: choose optimizer


In [7]:
from tqdm.notebook import tqdm


def estimated_label(params, data):
    return QCNN_circuit(params, data) # TODO: customize your extimated label

writer = SummaryWriter(log_dir='./runs')

for epoch in tqdm(range(EPOCHS)):
    for i, data in enumerate(training_loader):
        inputs, labels = data
        optimizer.zero_grad()
        avg_loss = torch.tensor(0, dtype=torch.float64)
        for input, label in zip(inputs, labels):
            output = estimated_label(params, inputs.to(torch.float64))
            avg_loss+=loss_fn(output, label.to(torch.float64))/len(labels)
        avg_loss.backward()        
        optimizer.step()

    running_vloss = 0.0
    for i, vdata in enumerate(test_loader):
        vloss = torch.tensor(0, dtype=torch.float64)
        vinputs, vlabels = vdata
        for vinput, vlabel in zip(vinputs, vlabels):
            voutput = estimated_label(vinput)
            vloss += loss_fn(voutput, vlabel)/len(vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and test
    writer.add_scalars('Training vs. test Loss',
                    { 'Training' : avg_loss, 'test' : avg_vloss },
                    epoch)
    writer.flush()
        # print('{}: {}'.format(i, loss.item()))


  0%|          | 0/10 [00:00<?, ?it/s]

0: 24.7500002332997
1: 13.5000002177464
2: 47.25000041993947
3: 23.250000021481377
4: 41.74999976821146
5: 32.49999995297021
6: 29.750000324013147
7: 53.24999994488692
8: 42.50000018718616
9: 59.00000036502385
10: 27.750000234582593
11: 38.2499997923805
12: 24.500000160207886
13: 59.00000053904148
14: 20.499999965759713
15: 45.74999997208993
16: 25.49999998078195
17: 22.499999933979094
18: 27.249999603124607
19: 46.50000002804726
20: 19.49999996859499
21: 29.249999896199093
22: 24.99999999073185
23: 16.24999985882196
24: 17.98040582346203
25: 27.750000065261204
26: 41.250000119399274
27: 26.25000027985385
28: 15.999999979943233
29: 28.75000028972165
30: 24.25000006810425
31: 27.000000342834717
32: 13.249999909091407
33: 27.750000102898433
34: 9.749999944863967
35: 48.499999675656625
36: 7.500000027799139
37: 36.463093227315646
38: 20.999999963770804
39: 37.50000021710897
40: 22.499999865831533
41: 56.49999987691088
42: 23.00000016650927
43: 0.7500000332837489
44: 40.250000033797434
45:

KeyboardInterrupt: 